In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
import pickle
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [2]:
random_seed = 0
random.seed(random_seed)


In [3]:
# reading the data from the given raw file
raw_dat = pd.read_csv('DSDataLastThreeMonths.csv')
# removing null values for further analysis
raw_dat = raw_dat.fillna(raw_dat.mean()).reset_index()
data2 = raw_dat.copy()


In [4]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6697 entries, 0 to 6696
Data columns (total 15 columns):
index            6697 non-null int64
CASTNO           6697 non-null object
HM_WT            6697 non-null int64
AIM_S            6697 non-null float64
HM_S             6697 non-null float64
HM_C             6697 non-null float64
HM_SI            6697 non-null float64
HM_TI            6697 non-null float64
HM_MN            6697 non-null float64
CAC2             6697 non-null int64
MG               6697 non-null int64
HM_TEMP          6697 non-null float64
CAC2_INJ_TIME    6697 non-null int64
MG_INJ_TIME      6697 non-null int64
DS_S             6697 non-null float64
dtypes: float64(8), int64(6), object(1)
memory usage: 784.9+ KB


In [5]:
mean = data2.loc[:,'HM_WT':'MG_INJ_TIME'].mean()
print('mean over whole data','\n',mean)
data2.loc[:,'HM_WT':'MG_INJ_TIME'] -= mean
sigma = raw_dat.loc[:,'HM_WT':'MG_INJ_TIME'].pow(2).mean()
sigma = sigma.pow(0.5)
# print(sigma)
data2.loc[:,'HM_WT':'MG_INJ_TIME']/=sigma

mean over whole data 
 HM_WT             165.411229
AIM_S               0.004969
HM_S                0.039572
HM_C                4.562660
HM_SI               0.612587
HM_TI               0.066564
HM_MN               0.042735
CAC2              354.522771
MG                 50.390623
HM_TEMP          1388.349688
CAC2_INJ_TIME       9.372555
MG_INJ_TIME         4.919068
dtype: float64


In [6]:
raw_dat = data2

In [7]:
# extracting the independent variable, X
x_data = raw_dat.loc[:,['HM_WT', 'AIM_S', 'HM_S', 'HM_C', 'HM_SI', 'HM_TI','HM_MN', 'CAC2', 'MG', 'HM_TEMP', 'CAC2_INJ_TIME', 'MG_INJ_TIME']]
# extracting the dependent variable, y
y_data = raw_dat.loc[:,'DS_S']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=0)

In [9]:
class tata_train(Dataset):

    def __init__(self, X_train, y_train):
        super().__init__()
        self.X_train = X_train
        self.y_train = y_train

    def __getitem__(self, idx):
        y = self.y_train.iloc[idx]
        z = int(y*1000)
        yo=0
        if z < 5:
            yo = 0 #2
        elif z < 10:
            yo = 1 #7
        elif z < 15:
            yo = 2 #12
        elif z < 20:
            yo = 3 #17
        elif z < 25:
            yo = 4 #22
        else:
            print('zzz', z)

        one_hot = np.zeros(5).astype('long')
        idx2 = int(yo)
        one_hot[idx2] = 1
        return torch.tensor( self.X_train.iloc[idx] ), torch.tensor( one_hot )
    def __len__(self):
        return len(self.X_train)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(

            nn.Linear(in_features = 12, out_features = 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            # nn.Sigmoid(),
            # nn.BatchNorm1d(30),

            nn.Linear(in_features = 20, out_features = 12),
            nn.BatchNorm1d(12),
            nn.ReLU(),
            # nn.Sigmoid(),
            # nn.BatchNorm1d(50),

            nn.Linear(in_features = 12, out_features = 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            # # # nn.Sigmoid(),
            # # nn.BatchNorm1d(50),
            
            nn.Linear(in_features = 20, out_features = 12),
            nn.BatchNorm1d(12),
            nn.ReLU(),
            # # # nn.Sigmoid(),
            # # nn.BatchNorm1d(30),

            nn.Linear(in_features = 12, out_features = 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            # # # nn.Sigmoid(),
            # # nn.BatchNorm1d(30),

            nn.Linear(in_features = 20, out_features = 12),
            nn.BatchNorm1d(12),
            nn.ReLU(),

            # nn.Linear(in_features = 30, out_features = 24),
            nn.Linear(in_features = 12, out_features = 5), # for modified output
)


    def forward(self, x):
        return self.linear(x)


In [11]:
train_ds = tata_train(X_train, y_train)

In [12]:
train_loader = DataLoader(train_ds, shuffle=True, batch_size=512)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Net().to(device)
print(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.03)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma = 0.1)

Net(
  (linear): Sequential(
    (0): Linear(in_features=12, out_features=20, bias=True)
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=12, bias=True)
    (4): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=12, out_features=20, bias=True)
    (7): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=20, out_features=12, bias=True)
    (10): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=12, out_features=20, bias=True)
    (13): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Linear(in_features=20, out_features=12, bias=True)
    (16): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [14]:
epochs = 50
train_losses = []
test_losses = []
for _ in range(epochs):
    scheduler.step()
    running_loss = 0.0
    for x , y in train_loader:
        # print(x, y)
        x = x.to(device)
        y = y.to(device)
        out = net(x)

        loss = criterion(out, torch.max(y,1)[1])
        running_loss += loss.item()*y.size(0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    running_loss = running_loss / len(train_loader.dataset)
    train_losses.append(running_loss)
    print('epoch ', _ ,' loss ', running_loss)

epoch  0  loss  1.212038121449857
epoch  1  loss  0.8423911449742327
epoch  2  loss  0.7520475155851217
epoch  3  loss  0.7297632903378827
epoch  4  loss  0.7095057532769253
epoch  5  loss  0.6976593416341967
epoch  6  loss  0.6940045778208208
epoch  7  loss  0.6899095164363638
epoch  8  loss  0.6882186070170936
epoch  9  loss  0.6865496848557499
epoch  10  loss  0.6781594115232603
epoch  11  loss  0.6768026422880294
epoch  12  loss  0.6761618227072183
epoch  13  loss  0.6718130607292475
epoch  14  loss  0.6761551261532801
epoch  15  loss  0.6794107558468969
epoch  16  loss  0.6735018596976557
epoch  17  loss  0.6749214312353147
epoch  18  loss  0.6695565333124043
epoch  19  loss  0.6597897135355725
epoch  20  loss  0.6573631047564747
epoch  21  loss  0.6494946308928944
epoch  22  loss  0.6497376348841578
epoch  23  loss  0.6489570381329945
epoch  24  loss  0.6458777018386341
epoch  25  loss  0.6472533648427554
epoch  26  loss  0.6461492824968992
epoch  27  loss  0.6448562133806495
epo

In [15]:
train_tensor = torch.from_numpy( np.array( X_train.loc[:,:]) ).float()
test_tensor = torch.from_numpy( np.array( X_test.loc[:,:] ) ).float()

In [16]:
#for moodified
d = { 0 : 2.0 , 1 : 7.0 , 2 : 12.0 , 3 : 17.0 , 4 : 22.0} #best config till now
print(train_tensor.size())
pred_train = net(train_tensor.to(device)).to('cpu').detach().numpy().argmax(axis=1).astype('int')


torch.Size([4486, 12])


In [17]:
pred_train = np.array([d[x] for x in pred_train])
print('pred_train\n', pred_train[:4])
pred_train /= 1000.0

pred_train
 [12.  7.  7.  7.]


In [18]:
#modified
pred_test = net(test_tensor.to(device)).to('cpu').detach().numpy().argmax(axis=1).astype('int')


In [19]:
pred_test = np.array([d[x] for x in pred_test])
print('pred_test\n', pred_test[:4])
pred_test /= 1000.0

pred_test
 [12.  2.  7.  7.]


In [20]:
#tolerance range
check = 0.003

In [21]:

# finding the error on the predictions
y_test = list(y_test)
y_train = list(y_train)

In [22]:
err_test = [x-y for x,y in zip(pred_test,y_test)]
err_train = [x-y for x,y in zip(pred_train,y_train)]

In [23]:
# finding the strike rates on the datasets
strike_rate_test = 100*sum([np.abs(x)<=check for x in err_test])/len(err_test)
strike_rate_train = 100*sum([np.abs(x)<=check for x in err_train])/len(err_train)

In [24]:

# printint the results
print("Test strike rate : {}\nTrain strike rate : {}".format(strike_rate_test,strike_rate_train))


Test strike rate : 87.92401628222524
Train strike rate : 88.80962995987517
